# 07 プランニング: AutoGen 連携サンプル (日本語版)

英語版 `07-planning-design/code_samples/07-autogen.ipynb` を基に、AutoGen を用いて
「ゴール文 → サブタスク分解(JSON) → 検証」という最小パイプラインを段階的に学べる形へ再構成します。

### 目的
ユーザーの自然言語入力によるゴールを受け取り、出力スキーマを先に定義したうえで LLM から構造化 JSON (TravelPlan) を取得します。
>出力スキーマを定義することで、LLM 任せの曖昧な出力を受け取らず、先に“決まった形の JSON フォーマットの出力だけ受け取る”と定義し、あとの処理で再実行しやすくします。
そして、Pydantic で検証してルーティング・実行・Re-plan（再実行）の基盤となるコードを実装します。

### 得られる成果物
- 先行スキーマ定義 (Task/Plan) による “出力スキーマ” の固定
- AutoGen クライアント(AzureAIChatCompletionClient) 初期化と最小プロンプト
- JSON / バリデーション / サブタスク列挙のパターン
- README の「タスク分解」「構造化出力」セクションとの対応理解

### セル構成
| セル | 種別 | 目的 | 主な要素 |
|------|------|------|----------|
| 1 | Markdown | 学習ゴール/構成提示 | 目的 / 成果物 / 対応表 |
| 2 | Code | インストール & インポート & スキーマ | %pip / Enum / Pydantic |
| 3 | Code | クライアント初期化 | トークン / endpoint / model / インスタンス化 |
| 4 | Code | 推論・抽出・検証 | プロンプト / 呼び出し / JSON / Validation |
| 5 | Markdown | 次ステップ | 発展 (ルーティング / Re-plan 等) |

### README との対応
- タスク分解: セル2で SubTask/Plan スキーマ、セル4で subtasks[] 生成と表示
- 構造化出力: セル4の JSON 取得 + Pydantic 検証
- （拡張）再計画(Re-plan): 未実装。Plan を引き回し差分生成セルを追加可能
- マルチエージェント連携: subtasks.assigned_agent を用いたルーティングの前段準備

### 実行手順
1. セル2: パッケージ インストール・スキーマ定義
2. セル3: クライアント初期化 (環境変数 GITHUB_TOKEN 必須)
3. セル4: goal 設定 / プロンプト実行 / JSON 抽出 / 検証 / プレビュー
4. 必要に応じて goal や SystemMessage を調整して再実行
5. セル5 の拡張案を参照し次段階 (ルーティング / Re-plan) へ


In [12]:
%pip install autogen-core autogen-ext[azure] pydantic --quiet

# =============================================
# セル2: インポート & 出力スキーマ定義
# 目的: 先にスキーマを固定して LLM 出力の形を“拘束”し後工程の安定性を確保する
# 成果物(1/4): 出力スキーマ固定 -> フラグ ARTIFACT_SCHEMA_DEFINED で可視化
# =============================================
import os, json
from pydantic import BaseModel, Field, ValidationError  # BaseModel 継承クラスの model_validate()/model_validate_json() はクラス定義に基づいて JSON 構造を検証する。
from enum import Enum
from typing import List

class AgentEnum(str, Enum):
    FlightBooking='flight_booking'
    HotelBooking='hotel_booking'
    CarRental='car_rental'
    ActivitiesBooking='activities_booking'
    DestinationInfo='destination_info'
    DefaultAgent='default_agent'

class TravelSubTask(BaseModel):
    task_details: str = Field(min_length=3, description='具体的で実行可能なサブタスク内容')
    assigned_agent: AgentEnum

class TravelPlan(BaseModel):
    main_task: str
    subtasks: List[TravelSubTask]
    is_greeting: bool

ARTIFACT_SCHEMA_DEFINED = True  # 成果物1 達成フラグ
print('[SCHEMA] TravelPlan / TravelSubTask 出力スキーマ定義完了 -> ARTIFACT_SCHEMA_DEFINED=True')


Note: you may need to restart the kernel to use updated packages.
[SCHEMA] TravelPlan / TravelSubTask 出力スキーマ定義完了 -> ARTIFACT_SCHEMA_DEFINED=True



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# =============================================
# セル3: クライアント初期化
# 目的: Azure AI Inference (GitHub Models) への接続を確立
# 成果物(2/4): クライアント初期化成功 -> ARTIFACT_CLIENT_INITIALIZED=True
# =============================================
from autogen_core.models import SystemMessage, UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
import re

# v0.4.7+ で model_info 必須フィールドが増加:
#   json_output / vision / function_calling / family / (structured_output)
# family: 同系統モデルのファミリー識別子。gpt-4o-mini -> 'gpt-4o'
# ここでは最小利用のため機能フラグは全て False。必要になったら True に変更。

token = os.getenv('GITHUB_TOKEN')
if not token:
    raise RuntimeError('GITHUB_TOKEN を .env に設定してください')

MODEL_NAME = 'gpt-4o-mini'
ENDPOINT = 'https://models.inference.ai.azure.com'
MODEL_INFO = {
    'family': 'gpt-4o',          # モデル系統 (必須) - mini/最新派生でもベース系列を指定
    'json_output': False,        # response_format='json_object' で強制するため False
    'vision': False,             # 画像入力なし
    'function_calling': False,   # ツール/関数呼び出し未使用
    'structured_output': False   # JSON Schema 指定を現時点では使わない
}

ARTIFACT_CLIENT_INITIALIZED = False
try:
    client = AzureAIChatCompletionClient(
        model=MODEL_NAME,
        endpoint=ENDPOINT,
        credential=AzureKeyCredential(token),
        model_info=MODEL_INFO
    )
    ARTIFACT_CLIENT_INITIALIZED = True
    print(f'[CLIENT] 初期化完了 model={MODEL_NAME} endpoint={ENDPOINT} -> ARTIFACT_CLIENT_INITIALIZED=True')
except ValueError as e:
    print('[CLIENT][ERROR] 初期化失敗:', e)
    m = re.search(r"Missing required field '([^']+)'", str(e))
    if m:
        missing = m.group(1)
        print(f"[HINT] model_info に '{missing}' キーを追加してください。")
        if missing == 'family':
            print("[HINT] 例: 'family': 'gpt-4o'")
    print('[HINT] エラー文に記載のキーを MODEL_INFO に追加し、セルを再実行。')
    raise


[CLIENT] 初期化完了 model=gpt-4o-mini endpoint=https://models.inference.ai.azure.com -> ARTIFACT_CLIENT_INITIALIZED=True


In [16]:
# =============================================
# セル4: 推論 -> JSON 抽出 -> 検証 -> プレビュー
# 目的: goal を最小プロンプトで分解し構造化出力を TravelPlan に検証
# 出力ログは セル1「得られる成果物」4項目をフラグ+理由で可視化
# =============================================
goal = '家族4人(子供2人)のシンガポール発メルボルン3日間旅行プランを作成'

# 案B: もう少し丁寧で日本語学習者にも意図が明確なシステムプロンプト
#  - 条件を箇条書きで明示
#  - JSON 以外は禁止を強調
#  - 各フィールドの意味/制約を説明
#  - ミニマルな形式サンプルを提示
try:
    system_prompt = SystemMessage(content=(
        "あなたは旅行計画を行うプランナーエージェントです。ユーザーのゴール文を読み、以下の要件に従ってタスクを分解し JSON を生成してください。\n\n"
        "【出力に関する絶対条件】\n"
        "- 出力は JSON オブジェクトのみ。説明文・コードブロック・余分な文字・コメントを一切含めない。\n"
        "- ルートのキーは main_task, subtasks, is_greeting の3つのみ。\n"
        "- main_task: 全体ゴールを一文で要約した文字列。\n"
        "- subtasks: 配列。各要素は {\"task_details\": str, \"assigned_agent\": <下記いずれか>} 形式。\n"
        "    * assigned_agent の選択肢: flight_booking | hotel_booking | car_rental | activities_booking | destination_info | default_agent\n"
        "    * task_details は後工程がそのまま行動できる具体的内容 (3文字以上)。\n"
        "- is_greeting: ユーザー入力が挨拶だけなら true, それ以外は false。\n"
        "- 余計なキーを追加しない。値が不要でも空文字や空配列ではなく意味のある最小情報を入れる。\n\n"
        "【形式サンプル】 (値は例。説明を含めずこの形だけを返す)\n"
        "{\"main_task\":\"家族旅行メルボルン計画\",\"subtasks\":[{\"task_details\":\"シンガポール発メルボルン往復航空券候補を取得\",\"assigned_agent\":\"flight_booking\"}],\"is_greeting\":false}\n\n"
        "上記条件を満たす最小十分な JSON 1つのみを返してください。"
    ), source='system')
    user_prompt = UserMessage(content=goal, source='user')
except Exception:
    # フォールバック (旧バージョン互換): source フィールド未対応ライブラリ用
    system_prompt = SystemMessage(content=(
        "あなたは旅行計画を行うプランナーエージェントです。ユーザーのゴール文を読み、以下の要件に従ってタスクを分解し JSON を生成してください。\n\n"
        "(source フィールド省略フォールバック)\n"
        "【出力条件】ルートキー: main_task, subtasks, is_greeting / JSON のみ / subtasks[].assigned_agent は指定の列挙から選択。"
    ))
    user_prompt = UserMessage(content=goal)

print('[STEP] Goal 受領 -> 推論開始')
import inspect, asyncio
resp = client.create(
    messages=[system_prompt, user_prompt],
    extra_create_args={'response_format': 'json_object'}
)
if inspect.iscoroutine(resp):
    try:
        resp = await resp
    except RuntimeError:
        loop = asyncio.get_event_loop()
        resp = loop.run_until_complete(resp)

raw_candidate = None
if hasattr(resp, 'content'):
    raw_candidate = resp.content
elif isinstance(resp, dict) and 'content' in resp:
    raw_candidate = resp['content']
elif hasattr(resp, 'choices'):
    try:
        choice0 = resp.choices[0]
        raw_candidate = getattr(getattr(choice0, 'message', choice0), 'content', None)
    except Exception:
        pass
raw = raw_candidate if isinstance(raw_candidate, str) else str(raw_candidate)
print('[RAW][TRUNCATED]', (raw or '')[:180].replace('\n',' '), '...')
if not raw:
    raise RuntimeError('レスポンスから content を抽出できませんでした。resp=' + repr(resp))

try:
    data = json.loads(raw)
except json.JSONDecodeError as e:
    raise RuntimeError(f'JSON デコード失敗: {e}\nRAW={raw}')

validation_ok = False
try:
    plan = TravelPlan.model_validate(data)
    validation_ok = True
except ValidationError as ve:
    print('[RESULT][NG] スキーマ検証失敗')
    print('[DETAIL]', ve)
    raise

from collections import Counter
agent_counts = Counter(t.assigned_agent.value for t in plan.subtasks)
agent_summary = ', '.join(f"{k}:{v}" for k, v in agent_counts.items())

print('\n[RESULT][OK] JSON -> TravelPlan バリデーション成功 (構造化出力適合)')
print(f'[DETAIL] main_task={plan.main_task}')
print(f'[DETAIL] subtasks={len(plan.subtasks)} (agent 内訳: {agent_summary})  is_greeting={plan.is_greeting}')

# ---- 成果物フラグ可視化 ----
# 1/4 スキーマ固定: ARTIFACT_SCHEMA_DEFINED
# 2/4 クライアント初期化: ARTIFACT_CLIENT_INITIALIZED
# 3/4 JSON/検証/列挙: validation_ok True & subtasks>=1
# 4/4 README 対応: subtasks フィールド存在 & JSON 構造整合

ARTIFACT_JSON_VALIDATED = validation_ok and len(plan.subtasks) >= 1
ARTIFACT_README_MAPPED = ('subtasks' in data) and validation_ok

rows = [
    ('1/4 出力スキーマ固定', ARTIFACT_SCHEMA_DEFINED, 'TravelPlan/TravelSubTask を Pydantic で定義済'),
    ('2/4 クライアント初期化', ARTIFACT_CLIENT_INITIALIZED, 'AzureAIChatCompletionClient インスタンス化成功'),
    ('3/4 JSON検証/列挙', ARTIFACT_JSON_VALIDATED, 'response_format=json_object -> JSON parse -> model_validate OK'),
    ('4/4 README対応', ARTIFACT_README_MAPPED, 'タスク分解(subtasks) + 構造化出力(JSON) を確認')
]
print('\n[ARTIFACTS] 達成状況')
for title, flag, reason in rows:
    status = '✅' if flag else '❌'
    print(f'  {status} {title:<16} : {reason}')

print('\n[PLAN] サブタスク一覧:')
for i, t in enumerate(plan.subtasks, 1):
    print(f'  - #{i:02d} [{t.assigned_agent.value}] {t.task_details}')


[STEP] Goal 受領 -> 推論開始
[RAW][TRUNCATED] {"main_task":"家族4人のシンガポール発メルボルン3日間旅行プラン作成","subtasks":[{"task_details":"シンガポール発メルボルン往復航空券候補を取得","assigned_agent":"flight_booking"},{"task_details":"メルボルンの宿泊施設を探す","assigned_agent": ...

[RESULT][OK] JSON -> TravelPlan バリデーション成功 (構造化出力適合)
[DETAIL] main_task=家族4人のシンガポール発メルボルン3日間旅行プラン作成
[DETAIL] subtasks=4 (agent 内訳: flight_booking:1, hotel_booking:1, activities_booking:1, destination_info:1)  is_greeting=False

[ARTIFACTS] 達成状況
  ✅ 1/4 出力スキーマ固定     : TravelPlan/TravelSubTask を Pydantic で定義済
  ✅ 2/4 クライアント初期化    : AzureAIChatCompletionClient インスタンス化成功
  ✅ 3/4 JSON検証/列挙    : response_format=json_object -> JSON parse -> model_validate OK
  ✅ 4/4 README対応     : タスク分解(subtasks) + 構造化出力(JSON) を確認

[PLAN] サブタスク一覧:
  - #01 [flight_booking] シンガポール発メルボルン往復航空券候補を取得
  - #02 [hotel_booking] メルボルンの宿泊施設を探す
  - #03 [activities_booking] メルボルンでの子供向けアクティビティを選定
  - #04 [destination_info] メルボルンの観光スポットについて情報を集める
[RAW][TRUNCATED] {"main_task":"家族4人のシンガポール発メルボルン3日間旅行プラン作成

### 他ノート (Azure OpenAI / Semantic Kernel) との目的を比較
| 目的 | Notebook | 観察ポイント |
|------|--------------------|--------------|
| フォールバック正規化 | `07-azure-openai.ipynb` | RAW 出力整形・再試行戦略 |
| プロンプト実験 | `07-azure-openai.ipynb` | 専用編集セル設計 |
| 関数再利用/拡張 | `07-semantic-kernel.ipynb` | create_function_from_prompt 利用 |
| サービス抽象化 | `07-semantic-kernel.ipynb` | Kernel + Service レイヤ |
| 最短動作確認 | `07-autogen.ipynb` | シンプルさ/行数 |
> 各 Notebook はフォルダパス `translations\ja\07-planning-design\code_samples` 以下に配置されています。

推奨学習フロー:
1. 本ノートで“最短で動く計画生成”を理解し、「スキーマ → JSON 出力 → 検証」の骨格を把握。
2. Azure OpenAI `07-azure-openai.ipynb`  ノートでプロンプト編集専用セルとフォールバック正規化コメントを比較し、堅牢化ポイントを確認。
3. さらに Semantic Kernel ノート `07-semantic-kernel.ipynb` で `create_function_from_prompt` による関数化を見て、再利用/差し替え戦略を理解。
4. 統合ノートを自作し: AutoGen で素早く試作 → Azure OpenAI 版の正規化/検証パターンを移植 → Semantic Kernel 形式で関数化し複数ゴールをバッチ処理 or サービス化。

> 次にやると良い実装例: `def generate_plan(goal: str) -> TravelPlan:` を抽出し、例外時に再プロンプト (max_retries) & バリデーション失敗統計を返す。